In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import enchant 
import nltk
from nltk.corpus import words
import re
import html
import re
dict_en = enchant.Dict("en_US")

In [3]:
# job_titles = pd.read_csv("job_title_need_clean_2.csv")

# Lower case and remove unicode job title

In [12]:
# job_titles['lower_case'] = ''
# job_titles['remove_unicode'] = ''
# for index, row in job_titles.iterrows():
#     job_title_lower_case = str.lower(row['c_employment_title'])
#     job_title_remove_unicode = html.unescape(job_title_lower_case)
    
#     job_titles.set_value(index, 'lower_case', job_title_lower_case)
#     job_titles.set_value(index, 'remove_unicode', job_title_remove_unicode)

In [11]:
# job_titles.to_csv("job_title_lower_case_and_remove_unicode.csv")

# Fixed spelling case

In [28]:
job_titles_step_3 = pd.read_csv("job_title_lower_case_and_remove_unicode.csv")
job_titles_step_3['fix_spelling'] = ''
job_titles_step_3['fix_spelling_score'] = 0
job_titles_step_3 = job_titles_step_3.drop('Unnamed: 0', 1)
words_not_in_en = pd.read_csv("words_not_in_en.csv")
words_not_in_en = words_not_in_en.drop('Unnamed: 0', 1)
job_titles_step_3.head()

,c_employment_title,lower_case,remove_unicode,fix_spelling,fix_spelling_score
0,Web Developer,web developer,web developer,,0
1,Web Developer,web developer,web developer,,0
2,Web Developer,web developer,web developer,,0
3,Web Master,web master,web master,,0
4,Web Developer,web developer,web developer,,0


### Check the title with all words are correct

In [29]:
def remove_special_character(word):
    result = ''.join(e for e in word if e.isalnum() or e == ' ')
    result = re.sub('[!@#$%^&*()]', '', result)
    return result

In [30]:
total = len(job_titles_step_3)
for index, row in job_titles_step_3.iterrows():
        
    if index % 10000 == 0:
        print(index, '/', total)
    
    job_title = remove_special_character(row['remove_unicode'])
    
    is_correct = True
    for word in job_title.split():
        if not dict_en.check(word):
            is_correct = False
            break
            
    for word in remove_special_character(row['c_employment_title']).split():
        if word.isupper() == True:
            is_correct = False
            # Run only 1 time for create new word in dictionary
#             word = str.lower(word)
#             new_words = words_not_in_en[words_not_in_en.word == word].head(1)
            
#             if len(new_words) == 0:
#                 suggest_word = str.lower(dict_en.suggest(word)[0])
#                 score = fuzz.ratio(word, suggest_word)
#                 words_not_in_en.loc[len(words_not_in_en)] = [word, 1, suggest_word, score]
#             else:
#                 frequency = new_words.get_value(new_words.index[0], 'frequency') + 1 
#                 words_not_in_en.set_value(new_words.index[0], 'frequency', frequency)
            break
            
            
    if is_correct:
        # All words in job_title are correctly spelling
        job_titles_step_3.set_value(index, 'fix_spelling', job_title)
        job_titles_step_3.set_value(index, 'fix_spelling_score', 1000) 


0 / 199665
10000 / 199665
20000 / 199665
30000 / 199665
40000 / 199665
50000 / 199665
60000 / 199665
70000 / 199665
80000 / 199665
90000 / 199665
100000 / 199665
110000 / 199665
120000 / 199665
130000 / 199665
140000 / 199665
150000 / 199665
160000 / 199665
170000 / 199665
180000 / 199665
190000 / 199665


In [31]:
total = len(job_titles_step_3)
for index, row in job_titles_step_3.iterrows():
    if index % 10000 == 0:
        print(index, '/', total)
        
    if row['fix_spelling_score'] < 1000:
        suggest_title = remove_special_character(row['remove_unicode'])
        original_title = remove_special_character(row['remove_unicode'])
        
        
        for word in original_title.split():
            wrong_words = words_not_in_en[words_not_in_en.word == word].head(1)
            if (len(wrong_words) == 1):
                
                suggest_word = wrong_words.get_value(wrong_words.index[0], 'suggest_word')
                wrong_word = wrong_words.get_value(wrong_words.index[0], 'word')
                score = wrong_words.get_value(wrong_words.index[0], 'score')
                
                
                if score >= 95:
                    suggest_title = suggest_title.replace(wrong_word, suggest_word)
        
        score = fuzz.ratio(original_title,suggest_title)
        job_titles_step_3.set_value(index, 'fix_spelling', suggest_title)
        job_titles_step_3.set_value(index, 'fix_spelling_score', score) 

0 / 199665
10000 / 199665
20000 / 199665
30000 / 199665
40000 / 199665
50000 / 199665
60000 / 199665
70000 / 199665
80000 / 199665
90000 / 199665
100000 / 199665
110000 / 199665
120000 / 199665
130000 / 199665
140000 / 199665
150000 / 199665
160000 / 199665
170000 / 199665
180000 / 199665
190000 / 199665


In [32]:
job_titles_step_3.to_csv("job_titles_step_3.csv")